Notebook flow

1. import libraries
2. define name variations that might be in sheet and map to standard last names 
3. create counting function that will: 
 - sum service weeks - name in any of these columns: 
     Floor A
     Floor B
     ICU low
     ICU high
     Burn
     
  - sum night calls total which means name shows up in column Call
      
  - sum weekend calls - name is in column Call where day of week is Fri, Sat or Sun
  
  - sums ICU weekends (Sat/Sun) where 2 days counts as 1 weekend 
  
  - comibine the above into a CallCounts dataframe
  
4. create a for loop that allows looping and summing over multiple months, with appropriate file output names 


In [ ]:
# Import libraries
import numpy as np
import pandas as pd


### Just below, change the MonthYr names to match the csv file inputs for the months of schedule you want examine

Format is 'MonthnameYYYY'

In [ ]:
MonthYrs = ['April2020', 'May2020']

In [ ]:
#define name perumatations that are likely to appear in schedule sheet
# Map all to just last name
# Fictional names used here ... 


names = { 
    'HP': 'Potter',
    'Harry': 'Potter',
    
    'HG':'Granger',
    'Hermione':'Granger',
    
    'RW':'Weasley',
    'Ron':'Weasley',
    
    'AD':'Dumbledore',
    'Albus':'Dumbledore',
    
    'RH':'Hagrid',
    'Rubeus':  'Hagrid',
    
    'MG':'McGonagall',
    'Minerva': 'McGonagall',
    
    'SS' : 'Snape',
    'Severus': 'Snape'}
 

In [ ]:
#Define what counts as weekends

wknd_srvc = ['Fri', 'Sat','Sun']  # Fri night, Sat, Sun count as wknds call counts    
wknd_icu = ['Sat', 'Sun']   #Sat/Sun only count as wknds for ICU (day shift only)

Define function that counts calls one month at a time:

In [ ]:

def count_calls(MonthYr):
    #Take input "MonthYr" eg "April2020" and read in csv file of that name in same directory
    path = '{}.csv'.format(MonthYr)
    sched = pd.read_csv(path)
    
    #replace name abbreviations/variations with last names, using mappings defined above 
    sched.replace(names,inplace=True)

    #Count times name shows up in call column, this includes weekends 
    callnightsTOT = sched.Call.value_counts() 
    
    #count times name shows up in call column, weekdays only 
    callwknt = sched.Call[~sched.Day.isin(wknd_srvc)].value_counts()

    #Count times name shows up in call column on weekends only (incl Fri, Sat and Sun nights; Sat Sun assumed 24hr)
    callwknds = sched.Call[sched.Day.isin(wknd_srvc)].value_counts()

    #Count days name shows up in A or B service column 
    #Calc number service weeks -> 5 days = 1 week 
    A = sched.Floor_A.value_counts()
    B = sched.Floor_B.value_counts()
    FloorDays= A.add(B, fill_value=0)
    FloorWks = FloorDays/5
    
    #Count days name shows up in ICU_low or ICU_high service column 
    #Calc number ICU weeks -> 5 days = 1 week 
    ICU_low = sched[~sched.Day.isin(wknd_icu)].ICU_low.value_counts()
    ICU_high = sched[~sched.Day.isin(wknd_icu)].ICU_high.value_counts()
    ICUdays= ICU_low.add(ICU_high, fill_value=0)
    ICUwks = ICUdays/5

    #Count weekends in ICU -> 2 days = 1 ICU weekend
    wknd_ICU = (sched.ICU_high[sched.Day.isin(wknd_icu)].value_counts())/2


    #Burn has to be handled specially because it is the only service people may double-cover
    #  with other services, leading to an over-count of servic weeks
    #We will create a second ghost Burn column BurnSingle
   
    sched['BurnSingle']=sched.Burn

    #Where the Burn attending's name shows up in another service column that day, the name will
    #  be eliminated from the BurnSingle column and the cell changed to NaN
    sched.BurnSingle[(sched.Burn==sched.Floor_A) |
                (sched.Burn==sched.Floor_B)  |
                (sched.Burn==sched.ICU_low) |
                (sched.Burn==sched.ICU_high)]=np.NaN

    #Then only the BurnSingle column will be used to count total service weeks, to avoid counting 1 wk as 2
    BurnSingleDays = sched.BurnSingle[~sched.Day.isin(wknd_icu)].value_counts()
    BurnSingleWks=BurnSingleDays/5

    #But the burn week count itself comes from the Burn column 
    #Count days name shows up in Burn service column
    #Calc number burn weeks -> 5 days = 1 week
    BurnDays = sched.Burn[~sched.Day.isin(wknd_icu)].value_counts()
    BurnWks=BurnDays/5

    #Sum the above across providers to create the columns for the CallCounts df
    CallCounts = pd.concat([FloorWks,
                        ICUwks,
                        BurnWks,
                        BurnSingleWks,
                        callwknt,
                        callwknds,
                        wknd_ICU,
                        callnightsTOT],
                        axis = 1)

    #rename the columns logically (follow order as above)
    CallCounts.columns = ['FloorWks',
                       'ICUwks',
                       'BurnWks',
                       'BurnSingleWks',
                       'WkNightCalls',
                       'WkndCalls',
                       'Wknd_ICU',
                       'NightCalls-Total']

    #replace NaNs with 0
    CallCounts.replace(np.NaN, 0, inplace=True)

    #Sum the ICU and service weeks for a total service weeks column
    CallCounts['TotalSrvcWks']= (CallCounts.FloorWks+
                            CallCounts.ICUwks)

    #Sum the burn, ICU and service weeks for a service+burn weeks column
    CallCounts['Burn+Srvc']=(CallCounts.FloorWks+
                         CallCounts.ICUwks+
                         CallCounts.BurnSingleWks)

    #Sum the call and ICU weekends for total weekends worked
    CallCounts['TotalWknds']= (CallCounts.WkndCalls+
                           CallCounts.Wknd_ICU)

    #sort alphabetically         
    CallCounts.sort_index(axis=0, inplace = True)
    CallCounts

    #output to excel with correct month-date filename
    CallCounts.to_excel('{}_CallCounts.xlsx'.format(MonthYr))
    
    return CallCounts

In [ ]:
#this will use the set lsit of MonthYrs from start of notebook

#set the initial counting df equal to the count_calls function for the first month ('MonthYrs[0]')

initial = count_calls(MonthYrs[0])

#then loop over remaining months in the list, couting calls, outputting the df to excel, and
# adding each successive iteration to the 'Total' df
# when loop complete, output the 'Total' df named for MonthYrs that it spans
# and display the total call df 

for m in MonthYrs[1:]:
    Total = initial + count_calls(m)

Total.to_excel('Total{}-{}_CallCounts.xlsx'.format(MonthYrs[0],MonthYrs[-1]))
Total

Done!